This code is writen to find a ball in which we want to impose dissipation

I need to load that data, divide each component by its std and then plot the norm. It is as simple as that, and I already have all the codes that is needed for this.

In [1]:
import numpy as np
import sys
sys.path.append('./../')
import torch
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from ProcessFunctions import find_Aisv2,Find_a_i,find_Aisv2_onlyv
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy.interpolate import interp1d
import seaborn as sns 
import cte_eq

In [2]:
T_final=10500
Ntout=cte_eq.Ntout
Nxout=cte_eq.Nxout
drs=0.012
data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_v=PODmodes['U']
S_v=PODmodes['S']
# VT_v=PODmodes['VT']
q_bar_v=PODmodes['q_bar']
Nt2=127534 # This is the number os snapshots used to find the POD, it is fined in the RunForward2D.ipynb

In [3]:
# Loading POD modes


data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyontheta'+'drs'+str(drs)+".npz"

PODmodes=np.load(data_dir)
U_theta=PODmodes['U']
S_theta=PODmodes['S']
# VT_theta=PODmodes['VT']
q_bar_theta=PODmodes['q_bar']

In [ ]:
# Loading data from "/central/groups/astuart/hkaveh/Data/LearnROM/transfer/SampleSimulation_Tf_2D'+str(T_final_run)+"Nt="+str(cte_eq.Ntout)+"N_m"+str(N_m)+"coeff"+str(coeff)+"number"+str(index)"
T_final_run = 350 # each initial condition is simulated for 250 years
Nt=5     # it is recorded every Nt time stes
N_m_v=20 # number of modes that they have considered
N_m_theta=20 # number of modes that they have considered
coeff=1 # How the inital conditions are from the chaotic attractor
number=10 # Which one do you want to load
N_cut=2000
# loading time series:
N_m_load=30

X_full=np.empty((1,N_m_load*2))
Y_full=np.empty((1,N_m_load*2))


for number in range(1,100):
    data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/SampleSimulation_Tf_2D'+str(T_final_run)+"Nt="+str(cte_eq.Ntout)+"N_m"+str(N_m_load)+"coeff"+str(coeff)+"number"+str(number)+".npz"
    data_smaple=np.load(data_dir)

    V_ox=data_smaple['array1']
    theta_ox=data_smaple['array2']
    t_ox=data_smaple['array3']
    # removing data with probability p
    # xi=100
    # p_remove=np.tanh(xi*np.max(V_ox,axis=1))
    # Nt_sample=p_remove.shape[0]
    # print( "number of points in this member is:", Nt_sample)
    # keep_mask_sample=np.random.rand(Nt_sample) > p_remove
    # We work with log10 of V_ox and theta_ox:
    V_ox=np.log10(V_ox)
    theta_ox=np.log10(theta_ox)
    A_v,P_v=find_Aisv2_onlyv(U_v,V_ox,q_bar_v,N_m_load) # you dont need P in general, but I need it to check if I do everything correctly
    A_theta,P_theta=find_Aisv2_onlyv(U_theta,theta_ox,q_bar_theta,N_m_load)
    # concatenating the two arrays
    A=np.concatenate((A_v,A_theta),axis=1)
    X_onesimulation=A[:-1,:]         # current step
    Y_onesimulation=A[1:,:]-A[:-1,:] # next step minus the current step
    print("shape of X_onesimulation before is",X_onesimulation.shape)
    # X_onesimulation=X_onesimulation[keep_mask_sample[:-1]]
    # Y_onesimulation=Y_onesimulation[keep_mask_sample[:-1]]
    print("shape of X_onesimulation after is",X_onesimulation.shape)

    X_full=np.append(X_full,X_onesimulation,axis=0)
    Y_full=np.append(Y_full,Y_onesimulation,axis=0)
    print("number is " +str(number)+ " size of data in this simulation is " +str(X_onesimulation.shape[0])+" total size is " + str(X_full.shape[0]))

# removing the first row which is empty
X_full=X_full[1:,:]
Y_full=Y_full[1:,:]

# print(np.max(X_full))
print(np.max(Y_full))
# Loading the the numpy arrays Input and Output which are generated for imposing dissipiation far away from the attractor
# # commenting here:
# data_dir="/central/groups/astuart/hkaveh/Data/LearnROM/EllipticShell_radi_outer"+str(radi_outer)+"radi_inner"+str(radi_inner)+"PODappliedseperately.npz"

# data_smaple=np.load(data_dir)
# Input_X=data_smaple['Input']
# Output_Y=data_smaple['Output']
# # appending the Input_X to the X_full and Output_Y to the Y_full
# # appending the Input_X to the X_full and Output_Y to the Y_full, but append from the begining of the array
# print(np.max(Output_Y))
# X_full=np.append(Input_X,X_full,axis=0)
# Y_full=np.append(Output_Y,Y_full,axis=0)
# # untill here

# deleting some colomns from X_full and Y_full that are should not be included based on N_m_v and N_m_theta
# index of the columns that should be deleted
index_delete=np.array([])
if N_m_v<N_m_load:
    index_delete=np.append(index_delete,np.arange(N_m_v,N_m_load))
if N_m_theta<N_m_load:
    index_delete=np.append(index_delete,np.arange(N_m_load+N_m_theta,2*N_m_load))


# Convert the array elements to integers
index_delete = index_delete.astype(int)

X_full=np.delete(X_full,index_delete,axis=1)
Y_full=np.delete(Y_full,index_delete,axis=1)
print("index of the columns that are deleted are:")
print(index_delete)

print(X_full.shape)
print("size of the data is " +str(X_full.shape[0]))
# 50, 55, 59, 63, 72, 73,79,81

In [ ]:
plt.plot(X_onesimulation[:, 0])


In [ ]:
data_dir

In [ ]:
# Loading POD modes for V and Theta
N_m_v_load=30
N_m_theta_load=30
data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_v=PODmodes['U']
S_v=PODmodes['S']
# VT_v=PODmodes['VT']
q_bar_v=PODmodes['q_bar']
Sigma_v=np.diagonal(S_v)
Lambda_v=Sigma_v**2/Nt2
Lambda_v=np.atleast_2d(Lambda_v[:N_m_v])
Sigma_v=(np.sqrt(Lambda_v))


data_dir='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/MainSimulation2D_Tf'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyontheta'+'drs'+str(drs)+".npz"
PODmodes=np.load(data_dir)
U_theta=PODmodes['U']
S_theta=PODmodes['S']
# VT_theta=PODmodes['VT']
q_bar_theta=PODmodes['q_bar']
Sigma_theta=np.diagonal(S_theta)
# Nt2=PODmodes['VT'].shape[0]
Lambda_theta=Sigma_theta**2/Nt2
Lambda_theta=np.atleast_2d(Lambda_theta[:N_m_theta])
Sigma_theta=(np.sqrt(Lambda_theta))
# you need to save Sigma_theta and Sigma_v



In [ ]:
N_m_theta_load

In [ ]:
Sigma_v=Sigma_v[0,:N_m_v]
Sigma_theta=Sigma_theta[0,:N_m_theta]

In [ ]:
# just added
alpha_v=X_full[:,:N_m_v]
alpha_theta=X_full[:,N_m_v:]


In [ ]:
# check if this compatible with the thing.
print(Sigma_v.shape)
print(Sigma_theta.shape)
print(alpha_v.shape)
print(alpha_theta.shape)
print(Nt2)

In [ ]:
normalized_alpha_v=alpha_v/Sigma_v [np.newaxis,:]
normalized_alpha_theta=alpha_theta/Sigma_theta [ np.newaxis,:]

In [ ]:
normalized_alpha=np.append(normalized_alpha_v,normalized_alpha_theta,axis=1)

In [ ]:
print(normalized_alpha.shape)

In [ ]:
# plot the timeseries of the norm of the normalized_alpha
norm_normalized_alpha=np.linalg.norm(normalized_alpha,axis=1)
plt.figure(figsize=(10,5))
plt.plot(norm_normalized_alpha)
plt.title("Norm of the normalized_alpha")
plt.xlabel("iteration")
# plt.xlim([0,1500])
# plt.ylim([0,100])

plt.ylabel("Norm of the normalized_alpha")
plt.grid(True)
plt.show()


In [ ]:
print(norm_normalized_alpha.shape)

In [ ]:
# saving Sigma_v and Sigma_theta adding informtion like Ntout,Nxout and T_final in directory
directory='/central/groups/astuart/hkaveh/Data/LearnROM/transfer/'+'sigma'+str(T_final)+"Nt="+str(Ntout)+"Nx="+str(Nxout)+'PODonlyonV'+'drs'+str(drs)+".npz"
np.savez(directory,Sigma_v=Sigma_v,Sigma_theta=Sigma_theta,Nt2=Nt2)


In [ ]:
plt.plot(Sigma_v)


In [ ]:
plt.plot(Sigma_theta)

# Conclusion
The ball seems to have a radi of 12.